# Visual Tampering Detection Analysis

This notebook visualizes **why** a parcel is classified as tampered by showing:
1. Ground truth UV map patches
2. Detected image patches
3. Preprocessed patches (after applying compare methods)
4. Similarity scores for each surface
5. Decision tree classification

In [ ]:
import sys
from pathlib import Path

# Setup paths
ROOT = Path.cwd().parent.parent  # Adjust if needed
sys.path.append(ROOT.as_posix())

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

from src.tampering.compare import CompareType, apply_homogenization, METRICS
from src.tampering.parcel import PATCH_ORDER
from src.tampering.utils import get_side_surface_patches
from src.tampering.metrics import compute_msssim, compute_ssim, compute_hog, compute_mae

# Load similarity scores
df = pd.read_csv(ROOT / 'data' / 'misc' / 'simscores_validation.csv')
print(f"Loaded {len(df)} similarity score records")

## Step 1: Select a Parcel to Analyze

In [ ]:
# Show available parcels and their tampering status
parcel_summary = df.groupby('parcel_id').agg({
    'tampered': 'first',
    'tampering': 'first',
    'view': 'nunique'
}).reset_index()
parcel_summary.columns = ['parcel_id', 'is_tampered', 'tampering_type', 'num_views']
print("\nAvailable parcels:")
print(parcel_summary)

# Filter tampered parcels
tampered_parcels = parcel_summary[parcel_summary['is_tampered'] == True]
print(f"\nFound {len(tampered_parcels)} tampered parcels")
print(tampered_parcels)

In [ ]:
# SELECT YOUR PARCEL HERE
PARCEL_ID = 2  # Change this to any parcel ID you want to analyze

# Get views for this parcel (excluding ground truth uvmaps)
parcel_views = df[(df['parcel_id'] == PARCEL_ID) & (df['gt_keypoints'] == False)]['view'].unique()
print(f"\nParcel {PARCEL_ID} has {len(parcel_views)} views")

# Select first view for analysis
if len(parcel_views) > 0:
    selected_view = parcel_views[0]
    print(f"Analyzing view: {selected_view}")
    
    # Show tampering info
    view_data = df[(df['parcel_id'] == PARCEL_ID) & (df['view'] == selected_view)]
    print(f"\nTampering status: {view_data['tampered'].iloc[0]}")
    print(f"Tampering type: {view_data['tampering'].iloc[0]}")
else:
    print("No views found for this parcel!")

## Step 2: Load and Display UV Maps

In [ ]:
IMAGE_ROOT = ROOT / "data" / "tampar_sample"

# Load ground truth UV map
gt_uvmap_path = IMAGE_ROOT / "uvmaps" / f"id_{str(PARCEL_ID).zfill(2)}_uvmap.png"
gt_uvmap = cv2.imread(gt_uvmap_path.as_posix())
gt_uvmap = cv2.cvtColor(gt_uvmap, cv2.COLOR_BGR2RGB)

# Load detected UV map (from the selected view)
detected_uvmap_path = IMAGE_ROOT / selected_view
detected_uvmap = cv2.imread(detected_uvmap_path.as_posix())
detected_uvmap = cv2.cvtColor(detected_uvmap, cv2.COLOR_BGR2RGB)

# Display both UV maps
fig, axes = plt.subplots(1, 2, figsize=(16, 8))
axes[0].imshow(gt_uvmap)
axes[0].set_title('Ground Truth UV Map\n(Reference)', fontsize=14, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(detected_uvmap)
axes[1].set_title('Detected UV Map\n(Test Image)', fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print(f"Ground truth shape: {gt_uvmap.shape}")
print(f"Detected shape: {detected_uvmap.shape}")

## Step 3: Extract and Compare Individual Surface Patches

This shows each surface (top, left, center, right, bottom) side-by-side with similarity scores.

In [ ]:
# Extract patches from both UV maps
gt_patches = list(get_side_surface_patches(gt_uvmap))
detected_patches = list(get_side_surface_patches(detected_uvmap))

# Surface names (matching PATCH_ORDER)
surface_names = ['top', 'left', 'center', 'right', 'bottom']

print(f"Extracted {len(gt_patches)} surface patches")
print(f"Patch size: {gt_patches[0].shape}")

### Visualize Each Surface with Multiple Compare Methods

In [ ]:
def visualize_surface_comparison(surface_idx, surface_name, compare_type='plain'):
    """
    Visualize comparison for a single surface
    Shows: [GT patch] [Detected patch] [GT processed] [Detected processed] + scores
    """
    gt_patch = gt_patches[surface_idx]
    detected_patch = detected_patches[surface_idx]
    
    # Skip white patches
    if np.mean(gt_patch) >= 250 or np.mean(detected_patch) >= 250:
        print(f"Skipping {surface_name} - white patch")
        return
    
    # Apply preprocessing
    processed_gt, processed_det = apply_homogenization(
        compare_type, gt_patch, detected_patch
    )
    
    # Compute metrics
    msssim_score = compute_msssim(processed_gt.astype(np.float32), processed_det.astype(np.float32))
    ssim_score = compute_ssim(processed_gt.astype(np.float32), processed_det.astype(np.float32))
    hog_score = compute_hog(processed_gt.astype(np.float32), processed_det.astype(np.float32))
    mae_score = compute_mae(processed_gt.astype(np.float32), processed_det.astype(np.float32))
    
    # Get ground truth label for this surface
    surface_data = df[
        (df['parcel_id'] == PARCEL_ID) & 
        (df['view'] == selected_view) & 
        (df['sideface_name'] == surface_name) &
        (df['compare_type'] == compare_type)
    ]
    
    is_tampered = surface_data['tampered'].iloc[0] if len(surface_data) > 0 else False
    tampering_type = surface_data['tampering'].iloc[0] if len(surface_data) > 0 else ''
    
    # Create visualization
    fig, axes = plt.subplots(2, 2, figsize=(14, 14))
    
    # Original patches
    axes[0, 0].imshow(gt_patch.astype(int))
    axes[0, 0].set_title('Ground Truth\n(Reference)', fontsize=12, fontweight='bold')
    axes[0, 0].axis('off')
    
    axes[0, 1].imshow(detected_patch.astype(int))
    axes[0, 1].set_title('Detected\n(Test)', fontsize=12, fontweight='bold')
    axes[0, 1].axis('off')
    
    # Processed patches
    axes[1, 0].imshow(processed_gt.astype(int))
    axes[1, 0].set_title(f'After {compare_type}\n(Reference)', fontsize=12)
    axes[1, 0].axis('off')
    
    axes[1, 1].imshow(processed_det.astype(int))
    axes[1, 1].set_title(f'After {compare_type}\n(Test)', fontsize=12)
    axes[1, 1].axis('off')
    
    # Add title with scores
    title_color = 'red' if is_tampered else 'green'
    tampered_text = f"TAMPERED ({tampering_type})" if is_tampered else "NOT TAMPERED"
    
    fig.suptitle(
        f'Surface: {surface_name.upper()} - {tampered_text}\n'
        f'Compare Method: {compare_type}\n'
        f'MSSSIM: {msssim_score:.4f} | SSIM: {ssim_score:.4f} | HOG: {hog_score:.4f} | MAE: {mae_score:.4f}\n'
        f'(Higher SSIM/MSSSIM = more similar | Lower HOG/MAE = more similar)',
        fontsize=14,
        fontweight='bold',
        color=title_color
    )
    
    plt.tight_layout()
    plt.show()
    
    return {
        'msssim': msssim_score,
        'ssim': ssim_score,
        'hog': hog_score,
        'mae': mae_score
    }

### Compare All Surfaces with 'plain' method

In [ ]:
# Visualize each surface
for i, surface_name in enumerate(surface_names):
    print(f"\n{'='*60}")
    print(f"Analyzing surface: {surface_name.upper()}")
    print('='*60)
    visualize_surface_comparison(i, surface_name, compare_type='plain')

### Try Different Compare Methods on a Specific Surface

In [ ]:
# Select a surface to analyze with different methods
SURFACE_TO_ANALYZE = 'center'  # Change to 'top', 'left', 'right', 'bottom'
surface_idx = surface_names.index(SURFACE_TO_ANALYZE)

# Try all compare methods
compare_methods = ['plain', 'canny', 'laplacian', 'meanchannel']

for method in compare_methods:
    print(f"\n{'='*60}")
    print(f"Compare method: {method.upper()}")
    print('='*60)
    visualize_surface_comparison(surface_idx, SURFACE_TO_ANALYZE, compare_type=method)

## Step 4: Summary Heatmap - All Surfaces, All Methods

In [ ]:
# Get all scores for this view
view_scores = df[(df['parcel_id'] == PARCEL_ID) & (df['view'] == selected_view)]

# Create heatmaps for each metric
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle(f'Similarity Scores Heatmap - Parcel {PARCEL_ID}', fontsize=16, fontweight='bold')

metrics = ['msssim', 'ssim', 'hog', 'mae']
titles = [
    'MSSSIM (higher=similar)',
    'SSIM (higher=similar)',
    'HOG Distance (lower=similar)',
    'MAE (lower=similar)'
]

for idx, (metric, title) in enumerate(zip(metrics, titles)):
    ax = axes[idx // 2, idx % 2]
    
    # Pivot data
    pivot = view_scores.pivot_table(
        index='sideface_name',
        columns='compare_type',
        values=metric,
        aggfunc='mean'
    )
    
    # Plot heatmap
    im = ax.imshow(pivot, cmap='RdYlGn' if metric in ['msssim', 'ssim'] else 'RdYlGn_r', aspect='auto')
    
    # Labels
    ax.set_xticks(np.arange(len(pivot.columns)))
    ax.set_yticks(np.arange(len(pivot.index)))
    ax.set_xticklabels(pivot.columns, rotation=45, ha='right')
    ax.set_yticklabels(pivot.index)
    ax.set_title(title, fontsize=12, fontweight='bold')
    
    # Add values
    for i in range(len(pivot.index)):
        for j in range(len(pivot.columns)):
            value = pivot.iloc[i, j]
            if not np.isnan(value):
                ax.text(j, i, f'{value:.3f}', ha='center', va='center', fontsize=9)
    
    plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()

## Step 5: Identify Most Different Surface

In [ ]:
# Calculate average dissimilarity for each surface
surface_scores = view_scores.groupby('sideface_name').agg({
    'msssim': 'mean',
    'ssim': 'mean',
    'hog': 'mean',
    'mae': 'mean',
    'tampered': 'first',
    'tampering': 'first'
}).reset_index()

# Sort by MSSSIM (lower = more different)
surface_scores_sorted = surface_scores.sort_values('msssim')

print("\nSurface Similarity Ranking (least similar first):")
print("="*80)
for _, row in surface_scores_sorted.iterrows():
    tampered_indicator = "🔴 TAMPERED" if row['tampered'] else "🟢 OK"
    print(f"{row['sideface_name']:10s} | MSSSIM: {row['msssim']:.4f} | HOG: {row['hog']:.4f} | {tampered_indicator}")
    if row['tampered']:
        print(f"           | Tampering type: {row['tampering']}")

most_different = surface_scores_sorted.iloc[0]
print(f"\n⚠️  Most different surface: {most_different['sideface_name'].upper()}")
print(f"   MSSSIM: {most_different['msssim']:.4f}")
print(f"   Actually tampered: {most_different['tampered']}")

## Step 6: Visualize the Most Different Surface in Detail

In [ ]:
most_diff_surface = most_different['sideface_name']
most_diff_idx = surface_names.index(most_diff_surface)

print(f"Detailed analysis of most different surface: {most_diff_surface.upper()}")
print("\nTrying all compare methods:")

for method in ['plain', 'canny', 'laplacian', 'meanchannel']:
    print(f"\n{'='*60}")
    print(f"Method: {method}")
    print('='*60)
    visualize_surface_comparison(most_diff_idx, most_diff_surface, compare_type=method)

## Step 7: Compare True Positive vs False Positive (if available)

In [ ]:
# Find a true positive and false positive example
tp_views = df[(df['tampered'] == True) & (df['gt_keypoints'] == False)]['view'].unique()
fp_views = df[(df['tampered'] == False) & (df['gt_keypoints'] == False)]['view'].unique()

print(f"True positive views: {len(tp_views)}")
print(f"False positive views: {len(fp_views)}")

if len(tp_views) > 0 and len(fp_views) > 0:
    # Get average scores
    tp_scores = df[df['view'].isin(tp_views)].groupby('sideface_name')[['msssim', 'hog']].mean()
    fp_scores = df[df['view'].isin(fp_views)].groupby('sideface_name')[['msssim', 'hog']].mean()
    
    # Plot comparison
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    surfaces = tp_scores.index
    x = np.arange(len(surfaces))
    width = 0.35
    
    # MSSSIM comparison
    axes[0].bar(x - width/2, tp_scores['msssim'], width, label='True Positive (Tampered)', color='red', alpha=0.7)
    axes[0].bar(x + width/2, fp_scores['msssim'], width, label='False Positive (Not Tampered)', color='orange', alpha=0.7)
    axes[0].set_xlabel('Surface')
    axes[0].set_ylabel('MSSSIM Score')
    axes[0].set_title('MSSSIM: True Positive vs False Positive\n(lower = more different)', fontweight='bold')
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(surfaces)
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # HOG comparison
    axes[1].bar(x - width/2, tp_scores['hog'], width, label='True Positive (Tampered)', color='red', alpha=0.7)
    axes[1].bar(x + width/2, fp_scores['hog'], width, label='False Positive (Not Tampered)', color='orange', alpha=0.7)
    axes[1].set_xlabel('Surface')
    axes[1].set_ylabel('HOG Distance')
    axes[1].set_title('HOG: True Positive vs False Positive\n(higher = more different)', fontweight='bold')
    axes[1].set_xticks(x)
    axes[1].set_xticklabels(surfaces)
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\nInterpretation:")
    print("- True Positives: Should show LOW MSSSIM (< 0.5) and HIGH HOG (> 10) on tampered surfaces")
    print("- False Positives: Show moderate scores due to lighting/angle differences, not actual tampering")

## Conclusion

This notebook shows you:
1. **What patches are being compared** - Ground truth vs detected UV map patches
2. **How preprocessing affects comparison** - Different compare methods (plain, canny, laplacian, etc.)
3. **Why a surface is flagged as tampered** - Visual differences + quantitative similarity scores
4. **Which surface is most different** - Ranked by similarity metrics

### Key Insights:
- **Low MSSSIM/SSIM** (< 0.5) indicates visual differences
- **High HOG/MAE** (> 10 for HOG) indicates texture/structure differences
- **True tampering** shows consistent low scores across multiple compare methods
- **False positives** often due to lighting, perspective, or acquisition differences
